In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
ball = pd.read_csv("/Users/annadaugaard/Desktop/VFF/explore/labelled_match_ball.csv", index_col=0)
players = pd.read_csv("/Users/annadaugaard/Desktop/VFF/explore/labelled_match_players.csv", index_col=0)


In [ ]:
event_data = pd.read_csv("/Users/annadaugaard/Desktop/VFF/raw_data/sample_match_1/Sample_Game_1_RawEventsData.csv")
event_data_passes = event_data[event_data["Type"] == "PASS"]
event_data_passes["From"] = (event_data_passes["From"].astype(str).str.replace("Player", "", regex=True)).astype(int)
event_data_passes["To"] = (event_data_passes["To"].astype(str).str.replace("Player", "", regex=True)).astype(int)
event_data_passes["Start X"] = (event_data_passes["Start X"]).astype(float) * 106
event_data_passes["End X"] = (event_data_passes["End X"]).astype(float) * 106
event_data_passes["Start Y"] = (event_data_passes["Start Y"]).astype(float) * 68
event_data_passes["End Y"] = (event_data_passes["End Y"]).astype(float) * 68

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
# Calculate differences to compute speed
ball['dx'] = ball['ball_x'].diff()
ball['dy'] = ball['ball_y'].diff()
ball['dt'] = ball['time'].diff()

# Calculate speed (Euclidean distance per time difference)
ball['speed'] = np.sqrt(ball['dx']**2 + ball['dy']**2) / ball['dt']
# Drop intermediate calculation columns from the cleaned tracking data
ball.drop(columns=['dx', 'dy', 'dt'], inplace=True)


In [ ]:

# Compute z-scores for the 'speed' column (ignoring NaN values)
ball['z_score'] = zscore(ball['speed'], nan_policy='omit')

# Define outlier threshold (z-score > 3 are considered outliers)
threshold = 3
ball_clean = ball[abs(ball['z_score']) < threshold].copy()

# Remove corresponding rows in df1 based on matching times (assumes 'Start Time [s]' in df1 matches 'time' in ball)
df1_clean = event_data_passes[event_data_passes['Start Time [s]'].isin(ball_clean['time']) & event_data_passes['End Time [s]'].isin(ball_clean['time'])].copy()

# Drop intermediate calculation columns from the cleaned tracking data
ball_clean.drop(columns=['dx', 'dy', 'dt', 'z_score'], inplace=True)
ball = ball_clean

In [ ]:
ball["time"].diff()

In [ ]:
-2.681862/0.04

In [ ]:
ball["speed"].diff()

In [ ]:
ball["acceleration"] = ball["speed"].diff() / ball["time"].diff()

print(ball[["time", "ball_x", "ball_y", "speed", "acceleration"]])

In [ ]:

# Extract relevant data for plotting
time_speed = ball_clean['time'][0:1000]
speed_values = ball_clean['speed'][0:1000]

start_times = df1_clean['Start Time [s]'][0:10]
end_times = df1_clean['End Time [s]'][0:10]

# Plot speed over time
plt.figure(figsize=(10, 5))
plt.plot(time_speed, speed_values, label='Speed', color='black', linewidth=1.5)

# Mark start times with blue and end times with red
plt.scatter(start_times, np.interp(start_times, time_speed, speed_values), color='blue', label='Start Time', marker='o', s=50)
plt.scatter(end_times, np.interp(end_times, time_speed, speed_values), color='red', label='End Time', marker='o', s=50)

# Labels and legend
plt.xlabel('Time (s)')
plt.ylabel('Speed')
plt.title('Speed over Time with Start and End Markers')
plt.legend()
plt.grid(True)

# Show plot
plt.show()

In [ ]:
# Merge the ball coordinates into the players dataframe based on the time column.
# (Make sure that the time values in both dataframes match up correctly.)
players_df = players.merge(ball, on="time", how="left")

# Compute the Euclidean distance from the player (x, y) to the ball (ball_x, ball_y)
players_df["distance_to_ball"] = np.sqrt((players_df["x"] - players_df["ball_x"])**2 +
                                          (players_df["y"] - players_df["ball_y"])**2)

# For each time point, rank the players by distance (1 = closest)
players_df["distance_rank"] = players_df.groupby("time")["distance_to_ball"].rank(method="min")

prank_1 = players_df[players_df["distance_rank"] == 1]

prank_1 = prank_1[prank_1["distance_to_ball"] < 0.1]


In [ ]:
import seaborn as sns
# Create a density plot on the column 'my_column'
plt.figure(figsize=(10, 6))
sns.kdeplot(data=prank_1, x='distance_to_ball', fill=True, color='skyblue')
plt.title('Density Plot of my_column')
plt.xlabel('my_column')
plt.ylabel('Density')
plt.show()

In [ ]:
speed_values = ball_clean['speed'][0:1000]

In [ ]:
# Group by time and id, then count occurrences
grouped = prank_1[0:100].groupby(["time", "id"]).size().unstack(fill_value=0)
# Create a stacked bar plot
ax = grouped.plot(kind="bar", stacked=True, figsize=(15, 6))
plt.legend(title="Player ID", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd


df = pd.DataFrame({"time": prank_1["time"], "id": prank_1["id"], "team": prank_1["Team"]})
df = df.sort_values("time").reset_index(drop=True)

def resolve_ties_by_team(df):
    """Resolve ties at the same timestamp by checking previous and next team's alignment."""
    unique_times = df["time"].unique()
    resolved = []
    for i, t in enumerate(unique_times):
        candidates = df[df["time"] == t]
        if len(candidates) == 1:
            resolved.append(candidates.iloc[0])
        else:
            # tie
            if resolved:
                prev_team = resolved[-1]["team"]
            else:
                prev_team = None

            if i < len(unique_times) - 1:
                next_time = unique_times[i+1]
                next_candidates = df[df["time"] == next_time]
                next_team = next_candidates.iloc[0]["team"] if len(next_candidates) > 0 else None
            else:
                next_team = None

            chosen = None
            # 1) match both prev_team & next_team
            if prev_team and next_team:
                both = candidates[(candidates["team"] == prev_team) & (candidates["team"] == next_team)]
                if len(both) == 1:
                    chosen = both.iloc[0]
            # 2) match prev_team
            if chosen is None and prev_team:
                match_prev = candidates[candidates["team"] == prev_team]
                if len(match_prev) == 1:
                    chosen = match_prev.iloc[0]
            # 3) match next_team
            if chosen is None and next_team:
                match_next = candidates[candidates["team"] == next_team]
                if len(match_next) == 1:
                    chosen = match_next.iloc[0]
            # fallback
            if chosen is None:
                chosen = candidates.iloc[0]
            resolved.append(chosen)
    return pd.DataFrame(resolved).reset_index(drop=True)

def compress_consecutive_id(df):
    """Group consecutive rows with the same id into a single block with start/end times."""
    blocks = []
    current_block = None
    for _, row in df.iterrows():
        if current_block is None:
            current_block = {
                "id": row["id"],
                "team": row["team"],
                "start_time": row["time"],
                "end_time": row["time"]
            }
        else:
            if row["id"] == current_block["id"]:
                current_block["end_time"] = row["time"]
            else:
                blocks.append(current_block)
                current_block = {
                    "id": row["id"],
                    "team": row["team"],
                    "start_time": row["time"],
                    "end_time": row["time"]
                }
    if current_block:
        blocks.append(current_block)
    return pd.DataFrame(blocks)

def build_pass_events(blocks_df):
    """Create a pass event for each adjacent pair of blocks."""
    blocks_df = blocks_df.sort_values("start_time").reset_index(drop=True)
    events = []
    for i in range(len(blocks_df) - 1):
        events.append({
            "Start Time [s]": blocks_df.loc[i, "start_time"],
            "End Time [s]": blocks_df.loc[i+1, "start_time"],
            "From": blocks_df.loc[i, "id"],
            "To": blocks_df.loc[i+1, "id"]
        })
    return pd.DataFrame(events)

# 2) Resolve ties
df_resolved = resolve_ties_by_team(df)

# 3) Compress consecutive IDs
df_blocks = compress_consecutive_id(df_resolved)

# 4) Build pass events
df_passes = build_pass_events(df_blocks)
print(df_passes)

In [ ]:
df_passes.to_csv("my_dataframe.csv", index=False)

In [ ]:
from collections import Counter

def investigate_pairs_overlap_ignore_index(A1, A2, B1, B2):
    """
    Compare two sets of pairs (A1, A2) and (B1, B2) without considering index position.
    It returns summary metrics based solely on how many pairs in A are also found in B.
    
    Metrics returned include:
      - common_pairs: dictionary of pairs present in both (with overlapping counts)
      - common_count: total number of pairs from A that appear in B (using minimum count for each common pair)
      - totalA: total number of pairs in A
      - common_percentage: percentage of pairs in A that are also in B
      
    Parameters:
      A1, A2: lists representing the first set of pairs.
      B1, B2: lists representing the second set of pairs.
      
    Returns:
      summary (dict): Dictionary with keys "common_pairs", "common_count", "totalA",
                      and "common_percentage".
    """
    # Ensure all lists have the same length
    n = len(A1)
    if not (len(A2) == n and len(B1) == n and len(B2) == n):
        raise ValueError("All lists must have the same length.")
    
    # Create list of pairs from each set.
    pairs_A = [(a1, a2) for a1, a2 in zip(A1, A2)]
    pairs_B = [(b1, b2) for b1, b2 in zip(B1, B2)]
    
    # Count frequencies.
    counterA = Counter(pairs_A)
    counterB = Counter(pairs_B)
    
    # For each pair that exists in both sets, take the minimum count.
    common_keys = set(counterA.keys()).intersection(set(counterB.keys()))
    common_pairs = {key: min(counterA[key], counterB[key]) for key in common_keys}
    common_count = sum(common_pairs.values())
    
    totalA = sum(counterA.values())
    common_percentage = (common_count / totalA) * 100 if totalA > 0 else 0
    
    summary = {
        "common_count": common_count,
        "totalA": totalA,
        "common_percentage": common_percentage,
    }
    print(f"\nTotal pairs in A: {totalA}")
    print(f"Common Count: {common_count}")
    print(f"Common Percentage (of A): {common_percentage:.2f}%")
    
    return summary
  
investigate_pairs_overlap_ignore_index(df_passes["From"][0:799], df_passes["To"][0:799],event_data_passes["From"][0:799], event_data_passes["To"][0:799])


In [ ]:
event_data_passes

In [ ]:
import io
import imageio
import matplotlib.pyplot as plt
import numpy as np
from mplsoccer import Pitch

def create_video_of_events(df_players, start_time, end_time, video_filename="events_video.mp4", fps=2):
    """
    Creates a video from player positions between start_time and end_time.
    Each frame shows the pitch and player positions (with their ids annotated).
    
    Parameters:
        df_players (pd.DataFrame): DataFrame containing player positions. Expected columns: 
                                   "time", "x", "y", "id", and "Team".
        start_time (float): The starting time for the video clip.
        end_time (float): The ending time for the video clip.
        video_filename (str): Output video file name.
        fps (int): Frames per second.
    """
    # Create a pitch (soccer field)
    pitch = Pitch(line_color="black")
    
    # Get all unique time points within the desired interval.
    times = sorted(df_players["time"].unique())
    times = [t for t in times if start_time <= t <= end_time]

    frames = []
    
    # Define team colors (adjust colors as desired)
    unique_teams = df_players["Team"].unique()
    team_colors = {team: color for team, color in zip(unique_teams, ["cornflowerblue", "tomato"])}

    for t in times:
        fig, ax = pitch.draw(figsize=(8, 6))
        # Filter the DataFrame for the current time.
        current_positions = df_players[df_players["time"] == t].copy()
        current_positions["color"] = current_positions["Team"].map(team_colors)
        
        # Plot player positions.
        pitch.scatter(current_positions["x"], current_positions["y"],
                      ax=ax, edgecolor="black", facecolor=current_positions["color"], alpha=0.8)
        
        pitch.scatter(current_positions["ball_x"], current_positions["ball_y"],
                      ax=ax, edgecolor="black", facecolor="black", alpha=1)
        # Annotate each player with their id.
        for _, row in current_positions.iterrows():
            ax.text(row["x"], row["y"], str(row["id"]), color="black", weight="bold",
                    ha="center", va="center")
        
        ax.set_title(f"Time: {t:.2f} s")
        
        # Instead of using canvas.tostring_rgb, save the figure to a BytesIO buffer.
        buf = io.BytesIO()
        fig.savefig(buf, format="png", bbox_inches="tight")
        buf.seek(0)
        image = imageio.imread(buf)
        frames.append(image)
        plt.close(fig)
    
    # Write the frames to a video file using imageio.
    imageio.mimwrite(video_filename, frames, fps=fps)


In [ ]:
create_video_of_events(players_df, 15, 22, video_filename="events_video.mp4", fps=2)